In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [5]:
#if use CNN type 1, if use resnet 18 type 2 , resnet 34 is type 3

select = 1
if select == 1:
    import torch
    import torch.nn as nn

    class AudioClassifier(nn.Module):
        def __init__(self):
            super(AudioClassifier, self).__init__()
            self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
            self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
            self.fc1 = nn.Linear(128*32*21, 1024)
            self.fc2 = nn.Linear(1024, len(mlb.classes_))

        def forward(self, x):
            x = x.unsqueeze(1)  # Add an extra dimension for the single channel
            x = torch.relu(self.conv1(x))
            x = torch.max_pool2d(x, 2)
            x = torch.relu(self.conv2(x))
            x = torch.max_pool2d(x, 2)
            x = x.view(x.size(0), -1)  # Flatten the tensor
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x
    print('CNN')
if select == 2:
    import torch
    import torch.nn as nn
    from torchvision import models

    class AudioClassifier(nn.Module):
        def __init__(self, num_classes):
            super(AudioClassifier, self).__init__()
            self.resnet = models.resnet18(pretrained=True)
            
            # Modify first layer to take 1-channel input
            self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            
            # Modify last layer to have the correct number of output classes
            num_features = self.resnet.fc.in_features
            self.resnet.fc = nn.Linear(num_features, num_classes)

        def forward(self, x):
            x = x.unsqueeze(1)  # Add an extra dimension for the single channel
            x = self.resnet(x)
            return x

    model = AudioClassifier(num_classes=len(mlb.classes_))
if select == 3:
    import torch
    import torch.nn as nn
    from torchvision import models

    class AudioClassifier(nn.Module):
        def __init__(self, num_classes):
            super(AudioClassifier, self).__init__()
            self.resnet = models.resnet34(pretrained=True)
            
            # Modify first layer to take 1-channel input
            self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            
            # Modify last layer to have the correct number of output classes
            num_features = self.resnet.fc.in_features
            self.resnet.fc = nn.Linear(num_features, num_classes)

        def forward(self, x):
            x = x.unsqueeze(1)  # Add an extra dimension for the single channel
            x = self.resnet(x)
            return x

    model = AudioClassifier(num_classes=len(mlb.classes_))

In [6]:
import sounddevice as sd
import librosa
import numpy as np
import torch

def get_mel_spectrogram(y, sr=22050, n_fft=2048, hop_length=512, n_mels=128):
    # Compute the spectrogram
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    D_abs = np.abs(D)
    # Convert to Mel scale
    mel_spec = librosa.feature.melspectrogram(S=librosa.amplitude_to_db(D_abs), sr=sr, n_mels=n_mels)
    return mel_spec

# Load the trained model
model = AudioClassifier()

try:
    model.load_state_dict(torch.load('AI2.pth'))
except:
    try:
        model.load_state_dict(torch.load('AI2.pth', map_location=torch.device('cpu')))
    except:
        state_dict = torch.load('AI2.pth', map_location=torch.device('cpu'))
        new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.eval()

# Record audio
duration = 2  # seconds
fs = 44100  # Sample rate
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
sd.wait()  # Wait until recording is finished

# Preprocess audio
recording = np.squeeze(recording)  # Remove the singleton dimension
recording = librosa.resample(recording, orig_sr=fs, target_sr=22050)  # Resample the recording
mel_spec = get_mel_spectrogram(recording)  # Compute the Mel spectrogram

# Make prediction
mel_spec = torch.from_numpy(mel_spec).unsqueeze(0).float()  # Add two singleton dimensions at the beginning
outputs = model(mel_spec)
probs = torch.nn.functional.softmax(outputs, dim=1)
top_prob, top_label = torch.max(probs, dim=1)

#print(f"Predicted label: {le.inverse_transform([top_label.item()])[0]}")
#print(f"Probability: {top_prob.item() * 100:.2f}%")
# Define the threshold
threshold = 0.10

# Apply threshold to probabilities
high_prob_mask = probs > threshold

# Get labels for high-probability predictions
high_prob_labels = le.inverse_transform(torch.where(high_prob_mask)[1])

print(f"Predicted labels: {high_prob_labels}")


AttributeError: 'MultiLabelBinarizer' object has no attribute 'classes_'